In [ ]:
!pip install transformers torch tqdm

  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving Building Safer LLM Apps with LangChain Templates and NVIDIA NeMo Guardrails.txt to Building Safer LLM Apps with LangChain Templates and NVIDIA NeMo Guardrails.txt


In [ ]:
import os
from transformers import VitsModel, AutoTokenizer
import torch
import scipy.io.wavfile
import numpy as np
from tqdm import tqdm
from google.colab import files

def text_to_speech(input_file, output_file, model_name="facebook/mms-tts-eng", chunk_size=500, pause_duration=1.0):
    # Load model and tokenizer
    model = VitsModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Read the input file
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    # Split the text into paragraphs
    paragraphs = text.split('\n\n')

    # Function to process a single paragraph
    def process_paragraph(paragraph):
        chunks = [paragraph[i:i+chunk_size] for i in range(0, len(paragraph), chunk_size)]
        audio_chunks = []
        for chunk in chunks:
            inputs = tokenizer(chunk, return_tensors="pt")
            with torch.no_grad():
                output = model(**inputs).waveform
            audio_chunks.append(output.squeeze().numpy())
        return np.concatenate(audio_chunks)

    # Generate silence for pauses
    sample_rate = model.config.sampling_rate
    silence = np.zeros(int(sample_rate * pause_duration))

    # Process each paragraph and add pauses
    full_audio = []
    for paragraph in tqdm(paragraphs, desc="Processing paragraphs"):
        if paragraph.strip():  # Skip empty paragraphs
            paragraph_audio = process_paragraph(paragraph)
            full_audio.append(paragraph_audio)
            full_audio.append(silence)

    # Concatenate all audio
    full_audio = np.concatenate(full_audio)

    # Save the output as an audio file
    scipy.io.wavfile.write(output_file, rate=sample_rate, data=full_audio)

    print(f"Audio saved to {output_file}")

    # Download the generated audio file
    files.download(output_file)

# Example usage
input_file = "Building Safer LLM Apps with LangChain Templates and NVIDIA NeMo Guardrails.txt"  # This should match the name of the file you uploaded
output_file = "nemo-guardrails.wav"
text_to_speech(input_file, output_file, pause_duration=1.0)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.64k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/145M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

tokenizer_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/47.0 [00:00<?, ?B/s]

Processing paragraphs: 100%|██████████| 69/69 [02:05<00:00,  1.82s/it]

Audio saved to nemo-guardrails.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
from transformers import VitsModel, AutoTokenizer
import torch
import scipy.io.wavfile
import numpy as np
from tqdm import tqdm
from google.colab import files

def text_to_speech(input_file, output_file, model_name="facebook/mms-tts-eng", chunk_size=500):
    # Load model and tokenizer
    model = VitsModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Read the input file
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    # Split the text into chunks
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    # Generate speech for each chunk
    audio_chunks = []
    for chunk in tqdm(chunks, desc="Processing chunks"):
        inputs = tokenizer(chunk, return_tensors="pt")
        with torch.no_grad():
            output = model(**inputs).waveform
        audio_chunks.append(output.squeeze().numpy())

    # Concatenate all audio chunks
    full_audio = np.concatenate(audio_chunks)

    # Save the output as an audio file
    scipy.io.wavfile.write(output_file, rate=model.config.sampling_rate, data=full_audio)

    print(f"Audio saved to {output_file}")

    # Download the generated audio file
    files.download(output_file)

# Example usage
input_file = "test.txt"  # This should match the name of the file you uploaded
output_file = "output.wav"
text_to_speech(input_file, output_file)

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

Audio saved to output.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving test2.txt to test2.txt


In [ ]:
import os
from transformers import VitsModel, AutoTokenizer
import torch
import scipy.io.wavfile
import numpy as np
from tqdm import tqdm
from google.colab import files

def text_to_speech(input_file, output_file, model_name="facebook/mms-tts-eng", chunk_size=500):
    # Load model and tokenizer
    model = VitsModel.from_pretrained(model_name)
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Read the input file
    with open(input_file, 'r', encoding='utf-8') as file:
        text = file.read()

    # Split the text into chunks
    chunks = [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

    # Generate speech for each chunk
    audio_chunks = []
    for chunk in tqdm(chunks, desc="Processing chunks"):
        inputs = tokenizer(chunk, return_tensors="pt")
        with torch.no_grad():
            output = model(**inputs).waveform
        audio_chunks.append(output.squeeze().numpy())

    # Concatenate all audio chunks
    full_audio = np.concatenate(audio_chunks)

    # Save the output as an audio file
    scipy.io.wavfile.write(output_file, rate=model.config.sampling_rate, data=full_audio)

    print(f"Audio saved to {output_file}")

    # Download the generated audio file
    files.download(output_file)

# Example usage
input_file = "test2.txt"  # This should match the name of the file you uploaded
output_file = "output.wav"
text_to_speech(input_file, output_file)

Some weights of the model checkpoint at facebook/mms-tts-eng were not used when initializing VitsModel: ['flow.flows.0.wavenet.in_layers.0.weight_g', 'flow.flows.0.wavenet.in_layers.0.weight_v', 'flow.flows.0.wavenet.in_layers.1.weight_g', 'flow.flows.0.wavenet.in_layers.1.weight_v', 'flow.flows.0.wavenet.in_layers.2.weight_g', 'flow.flows.0.wavenet.in_layers.2.weight_v', 'flow.flows.0.wavenet.in_layers.3.weight_g', 'flow.flows.0.wavenet.in_layers.3.weight_v', 'flow.flows.0.wavenet.res_skip_layers.0.weight_g', 'flow.flows.0.wavenet.res_skip_layers.0.weight_v', 'flow.flows.0.wavenet.res_skip_layers.1.weight_g', 'flow.flows.0.wavenet.res_skip_layers.1.weight_v', 'flow.flows.0.wavenet.res_skip_layers.2.weight_g', 'flow.flows.0.wavenet.res_skip_layers.2.weight_v', 'flow.flows.0.wavenet.res_skip_layers.3.weight_g', 'flow.flows.0.wavenet.res_skip_layers.3.weight_v', 'flow.flows.1.wavenet.in_layers.0.weight_g', 'flow.flows.1.wavenet.in_layers.0.weight_v', 'flow.flows.1.wavenet.in_layers.1.wei

Audio saved to output.wav


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>